# Final Stage. Ensemble (앙상블)

개별적으로 튜닝된 **XGBoost, LightGBM, CatBoost** 모델의 예측 결과를 결합하여 최종 성능을 극대화합니다.
가장 단순하면서도 강력한 **가중 평균(Weighted Blending)** 방식을 사용합니다.

### 📋 앙상블 대상
1.  `submission_xgb_optuna.csv` (XGBoost)
2.  `submission_lgbm_optuna.csv` (LightGBM)
3.  `submission_cat_optuna.csv` (CatBoost)

### 🚀 산출물
* `final_submission_ensemble.csv` (최종 제출용)

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings

warnings.filterwarnings('ignore')

# 1. 파일 경로 설정
# 앞서 실행한 모델들의 결과 파일이 있는 곳
SUBMISSION_DIR = './' # 현재 폴더에 저장했다고 가정

# 파일명 리스트
files = {
    'xgb': 'submission_xgb_optuna.csv',
    'lgbm': 'submission_lgbm_optuna.csv',
    'cat': 'submission_cat_optuna.csv'
}

# 2. 결과 파일 로드
preds = {}
for model_name, filename in files.items():
    path = os.path.join(SUBMISSION_DIR, filename)
    if os.path.exists(path):
        print(f"📄 로드 완료: {filename}")
        preds[model_name] = pd.read_csv(path)
    else:
        print(f"파일 없음: {filename} (이 모델은 제외하고 진행합니다)")

# 3. 앙상블 (단순 평균 Averaging)
# 성능이 더 좋은 모델에 가중치를 더 줄 수도 있습니다. (예: xgb*0.4 + lgbm*0.3 + cat*0.3)
# 일단은 안전하게 1:1:1 평균으로 갑니다.

if len(preds) > 0:
    final_df = preds[list(preds.keys())[0]].copy()
    
    # 초기화
    final_df['end_x'] = 0
    final_df['end_y'] = 0
    
    for model_name, df in preds.items():
        final_df['end_x'] += df['end_x']
        final_df['end_y'] += df['end_y']
    
    # 나누기 (평균)
    final_df['end_x'] /= len(preds)
    final_df['end_y'] /= len(preds)
    
    # 4. 최종 보정 (Clipping) 및 저장
    final_df['end_x'] = np.clip(final_df['end_x'], 0, 105)
    final_df['end_y'] = np.clip(final_df['end_y'], 0, 68)
    
    save_path = 'final_submission_ensemble.csv'
    final_df.to_csv(save_path, index=False)
    
    print(f"\n[최종 완료] 앙상블 결과 저장됨: {save_path}")

else:
    print("병합할 파일이 하나도 없습니다. 앞 단계의 모델링 코드를 먼저 실행해주세요.")

📄 로드 완료: submission_xgb_optuna.csv
📄 로드 완료: submission_lgbm_optuna.csv
📄 로드 완료: submission_cat_optuna.csv

[최종 완료] 앙상블 결과 저장됨: final_submission_ensemble.csv
